In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
try:
    import mxnet as mx
except:
    sys.stderr.write("Cannot import mxnet.\n")
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.ndimage.interpolation import map_coordinates
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

Cannot import mxnet.
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
Cannot import mxnet.


In [3]:
from sklearn.externals import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import GradientBoostingClassifier 

# sys.path.append('/home/yuncong/csd395/xgboost/python-package')
try:
    from xgboost.sklearn import XGBClassifier
except:
    sys.stderr.write('xgboost is not loaded.')

In [3]:
# batch_size = 256
# model_dir_name = 'inception-bn-blue'
# model_name = 'inception-bn-blue'
# model, mean_img = load_mxnet_model(model_dir_name=model_dir_name, model_name=model_name, 
#                                    num_gpus=8, batch_size=batch_size)

In [4]:
dataset_settings = read_csv(DATASET_SETTINGS_CSV, header=0, index_col=0)
dataset_settings

,classes,network_model,stain,margins_um,num_sample_per_class,stacks,cell_features_used,comment,raw_image_dataset
dataset_id,,,,,,,,,
20,NaN,Inception-BN,nissl,200/500,1000,MD585,NaN,NaN,NaN
21,NaN,Inception-BN,nissl,200/500,1000,MD589,NaN,NaN,NaN
22,NaN,Inception-BN,nissl,200/500,1000,MD594,NaN,NaN,NaN
23,pos/surPos/surNoclass/neg,inception-bn-blue,nissl,500,1000,MD585,NaN,NaN,NaN
24,pos/surPos/surNoclass/neg,inception-bn-blue,nissl,500,1000,MD589,NaN,NaN,NaN
25,pos/surPos/surNoclass/neg,inception-bn-blue,nissl,500,1000,MD594,NaN,NaN,NaN
26,NaN,inception-bn-blue,nissl,NaN,1000,MD594/MD589/MD585,NaN,far negatives for Pn,NaN
27,NaN,inception-bn-blue,nissl,NaN,1000,MD585,NaN,far negatives for all structures,NaN
28,NaN,inception-bn-blue,nissl,NaN,1000,MD589,NaN,far negatives for all structures,NaN


In [10]:
classifier_settings = read_csv(CLASSIFIER_SETTINGS_CSV, header=0, index_col=0)
classifier_settings

,model,margin_um,train_set_id,neg_composition,sample_weighting,input_img_version,svm_id,comment,num_samples_per_class
classifier_id,,,,,,,,,
30,lr,500,20/21,neg_has_all_surround,NaN,cropped_gray,30.0,NaN,NaN
31,lr,500,21/22,neg_has_all_surround,NaN,cropped_gray,31.0,NaN,NaN
32,lr,500,20/22,neg_has_all_surround,NaN,cropped_gray,32.0,NaN,NaN
33,gb1,500,20/21,neg_has_all_surround,NaN,cropped_gray,33.0,NaN,NaN
34,gb1,500,21/22,neg_has_all_surround,NaN,cropped_gray,34.0,NaN,NaN
35,gb1,500,20/22,neg_has_all_surround,NaN,cropped_gray,35.0,NaN,NaN
36,gb1,500,23/24,neg_has_all_surround,NaN,cropped_gray,36.0,NaN,NaN
37,lr,500,23/24,neg_has_all_surround,NaN,cropped_gray,37.0,NaN,NaN
38,lr,500,23/24/27/28,neg_has_surround_and_negative,NaN,cropped_gray,38.0,NaN,NaN


In [11]:
for classifier_id in [116]:

    classifier_properties = classifier_settings.loc[classifier_id]

    margin_um = classifier_properties['margin_um']
    feature_classifier_model = classifier_properties['model']
    sample_weighting = classifier_properties['sample_weighting']
    neg_composition = classifier_properties['neg_composition']
    svm_id = int(classifier_properties['svm_id'])
    if not np.isnan(classifier_properties['num_samples_per_class']):
        num_samples_per_class = int(classifier_properties['num_samples_per_class'])
    else:
        num_samples_per_class = None
        
    # for structure in all_known_structures:
    for structure in ['5N',
     '6N',
     '7N',
     '7n',
     'Amb',
     'LC',
     'LRt',
     'Pn',
     'Tz',
     'VLL',
     'RMC',
     'SNC',
     'SNR',
     '3N',
     '4N',
     'Sp5I',
     'Sp5O',
     'Sp5C',
     'PBG',
     '10N',
     'VCA',
     'VCP',
     'DC',
     'AP',
     '12N',
     'RtTg',
     'SC',
     'IC']:

        # Determine which labels to load.

        structures_to_sample = [structure]
        # structures_to_sample = all_known_structures
        # structures_to_sample = ['Pn']

        negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]

        margins_to_sample = [margin_um]
        surround_positive_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix=surr_l) 
                                     for m in margins_to_sample
                                     for s in structures_to_sample 
                                     for surr_l in all_known_structures
                                     if surr_l != s]
        surround_noclass_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix='noclass') 
                                     for m in margins_to_sample
                                     for s in structures_to_sample]

        if neg_composition == 'neg_has_everything_else':
            labels_to_sample = structures_to_sample + negative_labels_to_sample
        elif neg_composition == 'neg_has_only_surround_noclass':
            labels_to_sample = structures_to_sample + surround_noclass_labels_to_sample + ['noclass']
        elif neg_composition == 'neg_has_all_surround':
            labels_to_sample = structures_to_sample + surround_positive_labels_to_sample + surround_noclass_labels_to_sample + ['noclass']
        elif neg_composition == 'neg_has_surround_and_negative':
            labels_to_sample = structures_to_sample + negative_labels_to_sample + surround_positive_labels_to_sample + surround_noclass_labels_to_sample
        else:
            raise Exception("neg_composition %s not recognized" % neg_composition)

        ######################

        training_set_ids = map(int, str(classifier_properties['train_set_id']).split('/'))
        training_features, training_addresses = load_datasets_bp(training_set_ids, labels_to_sample=labels_to_sample)

        ########################

        labels_found = training_features.keys()
        structures_found = set([convert_to_original_name(l) for l in labels_found 
                            if convert_to_original_name(l) in labels_found]) - {'noclass'}

        ######################

        print structure

        #############################
        ## Define Positive Samples ##
        #############################

        features_pos = training_features[structure]

        if num_samples_per_class is not None:
            features_pos = features_pos[np.random.choice(range( len(features_pos)), replace=False, size=(min(num_samples_per_class, len(features_pos)),))]    

        n_pos = len(features_pos)
        print n_pos, 'positive examples'

        #############################
        ## Define Negative Samples ##
        #############################

        if neg_composition == 'neg_has_only_surround_noclass':
            neg_classes = [convert_to_surround_name(structure, margin=margin_um, suffix='noclass')]
        elif neg_composition == 'neg_has_all_surround':
            neg_classes = [convert_to_surround_name(structure, margin=margin_um, suffix='noclass')]
            for surr_s in all_known_structures:
                c = convert_to_surround_name(structure, margin=margin_um, suffix=surr_s)
                if c in labels_found:
                    neg_classes.append(c)
        elif neg_composition == 'neg_has_everything_else':
            neg_classes = [structure + '_negative']
        elif neg_composition == 'neg_has_surround_and_negative':
            neg_classes = [convert_to_surround_name(structure, margin=margin_um, suffix='noclass')]
            for surr_s in all_known_structures:
                c = convert_to_surround_name(structure, margin=margin_um, suffix=surr_s)
                if c in labels_found:
                    neg_classes.append(c)
            neg_classes += [structure + '_negative']
        else:
            raise Exception('neg_composition %s is not recognized.' % neg_composition)

        features_neg = np.concatenate([training_features[neg_class] for neg_class in neg_classes])

        if num_samples_per_class is not None:
            features_neg = features_neg[np.random.choice(range( len(features_neg)), replace=False, size=(min(num_samples_per_class, len(features_neg)),))]

        n_neg = len(features_neg)
        print n_neg, 'negative examples'

        ###########################
        ## Define Sample Weights ##
        ###########################

        if sample_weighting == 'weighted':
            neg_distances = np.concatenate([distances_to_structures[neg_class][structure] for neg_class in neg_classes])

            sample_weights_neg = np.ones((n_neg,))
            sample_weights_neg[neg_distances > thresh] = diminishing(neg_distances[neg_distances > thresh])
            sample_weights = np.r_[np.ones((n_pos,)), sample_weights_neg]
        else:
            sample_weights = None

        ###########################################################################################

        train_data = np.r_[features_pos, features_neg]
        train_labels = np.r_[np.ones((features_pos.shape[0], )), 
                             -np.ones((features_neg.shape[0], ))]

        #     n_train = len(train_labels0)
        #     ri = np.random.choice(range(n_train), size=100, replace=False)
        #     train_data = train_data0[ri]
        #     train_labels = train_labels0[ri]

        #####################################
        ## Define feature_classifier_model ##
        #####################################

        if feature_classifier_model == 'lr':
            clf = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, 
                                     fit_intercept=True, intercept_scaling=1, class_weight=None, 
                                     random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', 
                                     verbose=0, warm_start=False, n_jobs=1)

        elif feature_classifier_model == 'lin_svc':
            clf = SVC(C=1.0, kernel='linear', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
                      probability=True, tol=0.001, cache_size=1000, max_iter=-1,
                  decision_function_shape=None, random_state=None)


        elif feature_classifier_model == 'lin_svc_calib':

            sv_uncalibrated = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, 
                                    C=1.0, multi_class='ovr', 
                                    fit_intercept=True, intercept_scaling=1, max_iter=100)
            clf = CalibratedClassifierCV(sv_uncalibrated)


        elif feature_classifier_model == 'xgb1':
            clf = XGBClassifier(max_depth=3, learning_rate=0.2, n_estimators=200, 
                                silent=False, objective='binary:logistic', nthread=-1, gamma=0, 
                                min_child_weight=20, max_delta_step=0, subsample=.8, 
                                colsample_bytree=.8, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, 
                                scale_pos_weight=1, base_score=0.5, seed=0, missing=None)
        
        elif feature_classifier_model == 'xgb2':
            clf = XGBClassifier(max_depth=5, learning_rate=0.2, n_estimators=100, 
                                silent=False, objective='binary:logistic')
            # 40s, 10,000 pos and 10,000 neg samples
            
        elif feature_classifier_model == 'gb1':
            clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.3, n_estimators=200, 
                                             subsample=1., criterion='friedman_mse', 
                                             min_samples_split=50, min_samples_leaf=20, 
                                             min_weight_fraction_leaf=0.0, max_depth=3, 
                                             min_impurity_split=1e-07, init=None, random_state=None, 
                                             max_features=None, verbose=1, max_leaf_nodes=None, 
                                             warm_start=False, presort='auto')

        elif feature_classifier_model == 'gb2':
            clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.3, n_estimators=100, 
                                             subsample=1., criterion='friedman_mse', 
                                             min_samples_split=50, min_samples_leaf=20, 
                                             min_weight_fraction_leaf=0.0, max_depth=5, 
                                             min_impurity_split=1e-07, init=None, random_state=None, 
                                             max_features=None, verbose=1, max_leaf_nodes=None, 
                                             warm_start=False, presort='auto')


        else:
            sys.stderr.write('Setting is not recognized.\n')

        t = time.time()    
        clf.fit(train_data, train_labels, sample_weight=sample_weights)
        sys.stderr.write('Fitting classifier: %.2f seconds\n' % (time.time() - t))

        clf_fp = DataManager.get_classifier_filepath(classifier_id=svm_id, structure=structure)
        create_parent_dir_if_not_exists(clf_fp)
        joblib.dump(clf, clf_fp)
        upload_to_s3(clf_fp)

        from sklearn.metrics import accuracy_score
        train_acc = accuracy_score(train_labels, clf.predict(train_data), normalize=True)
        print train_acc

5N
10000 positive examples
10000 negative examples


Fitting classifier: 1290.23 seconds


aws s3 cp "/shared/CSHL_classifiers/setting_116/classifiers/5N_clf_setting_116.dump" "s3://mousebrainatlas-data/CSHL_classifiers/setting_116/classifiers/5N_clf_setting_116.dump"
0.90045
6N
3784 positive examples
10000 negative examples


Fitting classifier: 258.36 seconds


aws s3 cp "/shared/CSHL_classifiers/setting_116/classifiers/6N_clf_setting_116.dump" "s3://mousebrainatlas-data/CSHL_classifiers/setting_116/classifiers/6N_clf_setting_116.dump"
0.96786128845
7N
10000 positive examples
10000 negative examples


...

Fitting classifier: 1639.78 seconds


aws s3 cp "/shared/CSHL_classifiers/setting_116/classifiers/LRt_clf_setting_116.dump" "s3://mousebrainatlas-data/CSHL_classifiers/setting_116/classifiers/LRt_clf_setting_116.dump"
0.8591
Pn
10000 positive examples
10000 negative examples


Fitting classifier: 1481.98 seconds


aws s3 cp "/shared/CSHL_classifiers/setting_116/classifiers/Pn_clf_setting_116.dump" "s3://mousebrainatlas-data/CSHL_classifiers/setting_116/classifiers/Pn_clf_setting_116.dump"


KeyboardInterrupt: 